## Question - Answering with PDF

- Document Loaders
- Chat Models
- Embeddings
- Vector Stores
- Retrieval-Augmented Generation

### Document Loaders

In [1]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../data/big-book-of-machine-learning-use-cases-2nd-edition.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

94


In [2]:
print(docs[0].page_content[0:100])
print(docs[0].metadata)

Big Book of Machine 
Learning Use Cases
A collection of technical 
blogs, including code 
samples an
{'source': '../data/big-book-of-machine-learning-use-cases-2nd-edition.pdf', 'page': 0}


In [3]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [4]:
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))

retriever = vectorstore.as_retriever()

/Users/dazzpool/Documents/Github/docuChatAI/venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/var/folders/kc/_ltsfdfs0c52y7zx5573skr40000gn/T/ipykernel_13746/2844637147.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))
/Users/dazzpool/Documents/Github/docuChatAI/venv/lib/python3.11/site-pa

In [5]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "What are the projects I should work on?"})

results

{'input': 'What are the projects I should work on?',
 'context': [Document(metadata={'page': 75, 'source': '../data/big-book-of-machine-learning-use-cases-2nd-edition.pdf'}, page_content='Notebook , and RasterFrames Notebook . Also, stay tuned for a new section in our \ndocumentation specifically for geospatial topics of interest.\n76\nEBOOK: BIG BOOK OF MACHINE LEARNING USE CASES — 2ND EDITION'),
  Document(metadata={'page': 62, 'source': '../data/big-book-of-machine-learning-use-cases-2nd-edition.pdf'}, page_content='DEFENSE AND INTEL\nReconnaissance, threat \ndetection, damage assessmentFINANCIAL SERVICES\nEconomic distribution, loan risk \nanalysis, predicting sales at \nretail, investments\nINFRASTRUCTURE\nTransportation planning, \nagriculture management, \nhousing developmentHEALTHCARE\nIdentifying disease epicenters, \nenvironmental impact on \nhealth, planning care\nENERGY\nClimate change analysis, energy \nasset inspection, oil discovery\nMaps leveraging geospatial data  \nar